## Interaktiivne Phi 3 Mini 4K Instruct vestlusrobot koos Whisper

### Sissejuhatus:
Interaktiivne Phi 3 Mini 4K Instruct vestlusrobot on tööriist, mis võimaldab kasutajatel suhelda Microsoft Phi 3 Mini 4K instruct demo abil teksti- või helisisendi kaudu. Vestlusrobotit saab kasutada mitmesugusteks ülesanneteks, näiteks tõlkimiseks, ilmateadete värskenduste saamiseks ja üldiseks teabe kogumiseks.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Loo oma Huggingface'i juurdepääsu token](https://huggingface.co/settings/tokens)

Loo uus token 
Sisesta uus nimi 
Vali kirjutamisõigused
Kopeeri token ja salvesta see turvalisse kohta


Järgmine Python-kood teeb kahte peamist ülesannet: impordib `os`-mooduli ja seab keskkonnamuutuja.

1. `os` mooduli importimine:
   - Python-i `os`-moodul pakub võimalust suhelda operatsioonisüsteemiga. See võimaldab teha erinevaid operatsioonisüsteemiga seotud toiminguid, näiteks pääseda juurde keskkonnamuutujatele, töötada failide ja kataloogidega jne.
   - Selles koodis imporditakse `os`-moodul `import`-lausungi abil. See lause teeb `os`-mooduli funktsionaalsuse kättesaadavaks praeguses Python-skriptis.

2. Keskkonnamuutuja seadmine:
   - Keskkonnamuutuja on väärtus, millele saavad ligi operatsioonisüsteemil töötavad programmid. See on viis salvestada konfiguratsiooniseadeid või muud teavet, mida saavad kasutada mitmed programmid.
   - Selles koodis luuakse uus keskkonnamuutuja, kasutades `os.environ` sõnastikku. Sõnastiku võti on `'HF_TOKEN'`, ja väärtus määratakse muutujast HUGGINGFACE_TOKEN.
   - Muutuja HUGGINGFACE_TOKEN on määratletud vahetult selle koodilõigu kohal ning sellele omistatakse stringiväärtus "hf_**************" kasutades `#@param` süntaksit. Seda süntaksit kasutatakse sageli Jupyter märkmikel, et võimaldada kasutaja sisestust ja parameetrite seadistamist otse märkmiku liideses.
   - Seades keskkonnamuutuja `'HF_TOKEN'`, saavad sellele ligi programmi muud osad või samal operatsioonisüsteemil töötavad teised programmid.

Kokkuvõttes impordib see kood `os`-mooduli ja seab keskkonnamuutuja nimega `'HF_TOKEN'` väärtusega, mis on määratud muutujas HUGGINGFACE_TOKEN.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Selle koodilõigu abil määratletakse funktsioon clear_output, mida kasutatakse praeguse lahtri väljundi tühjendamiseks Jupyter Notebookis või IPythonis. Lähme läbi koodi ja mõistame selle funktsionaalsust:

Funktsioon clear_output võtab ühe parameetri nimega wait, mis on loogiline väärtus. Vaikimisi on wait määratud väärtusele False. See parameeter määrab, kas funktsioon peaks enne tühjendamist ootama, kuni uus väljund on olemasoleva väljundi asendamiseks.

Funktsioon ise kasutatakse praeguse lahtri väljundi tühjendamiseks. Jupyter Notebookis või IPythonis, kui lahter genereerib väljundi, nagu näiteks ekraanile trükitud tekst või graafilised joonised, siis see väljund kuvatakse lahtri all. Funktsioon clear_output võimaldab selle väljundi tühjendada.

Funktsiooni implementatsioon ei ole koodilõigus esitatud, mida näitab ellipsis (...). Ellipsis tähistab kohatäitjat tegelikule koodile, mis täidab väljundi tühjendamise. Funktsiooni implementatsioon võib hõlmata suhtlust Jupyter Notebooki või IPythoni API-ga, et eemaldada olemasolev väljund lahtrist.

Kokkuvõttes pakub see funktsioon mugavat viisi praeguse lahtri väljundi tühjendamiseks Jupyter Notebookis või IPythonis, muutes kuvatud väljundi haldamise ja uuendamise interaktiivsete kodeerimisseansside ajal lihtsamaks.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Tehke tekstist kõne (TTS) Edge TTS teenuse abil. Läheme läbi vastavate funktsioonide implementatsioonid üks haaval:

1. `calculate_rate_string(input_value)`: See funktsioon võtab sisendi väärtuse ja arvutab TTS-hääle kiiruse stringi. Sisendi väärtus tähistab soovitud kõnekiirust, kus väärtus 1 tähendab normaalkiirust. Funktsioon arvutab kiiruse stringi, lahutades sisendväärtusest 1, korrutades tulemuse 100-ga ja määrates seejärel märgi sõltuvalt sellest, kas sisendväärtus on suurem või võrdne 1-ga. Funktsioon tagastab kiiruse stringi formaadis "{sign}{rate}".

2.`make_chunks(input_text, language)`: See funktsioon võtab parameetriteks sisendi teksti ja keele. See jagab sisendi teksti lõikudeks keele-spetsiifiliste reeglite alusel. Selles implementatsioonis, kui keel on "English", jagab funktsioon teksti igal tähel (".") ja eemaldab igast lõigust juhtiva ja lõpliku tühiku. Seejärel lisab ta igale lõigule punkti ja tagastab filtreeritud lõikude loendi.

3. `tts_file_name(text)`: See funktsioon genereerib TTS-audiofailile nime sisestatud teksti alusel. Ta teostab teksti peal mitmeid teisendusi: eemaldab lõppemispunkti (kui see on olemas), teisendab teksti väikesteks tähtedeks, lõikab juhtivad ja lõplikud tühikud ning asendab tühikud alakriipsudega. Seejärel lühendab teksti maksimaalselt 25 märgini (kui pikem) või kasutab kogu teksti, kui see on tühi. Lõpuks genereerib ta juhusliku stringi kasutades [`uuid`] moodulit ja kombineerib selle lühendatud tekstiga, et luua failinimi formaadis "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: See funktsioon liidab mitu audiofaili üheks audiofailiks. See võtab parameetriteks audiofailide teekondade loendi ja väljundteekonna. Funktsioon initsialiseerib tühja `AudioSegment` objekti nimega [`merged_audio`]. Seejärel iteriseerib ta läbi iga audiofaili teekonna, laadib audiofaili kasutades `AudioSegment.from_file()` meetodit `pydub`-raamatukogust ja lisab praeguse audiofaili [`merged_audio`] objektile. Lõpuks ekspordib ta liidetud audio määratud väljundteekonnale MP3-formaadis.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path): See funktsioon teostab TTS-operatsiooni kasutades Edge TTS teenust. See võtab parameetriteks tekstilõikude loendi, kõne kiiruse, hääle nime ja salvestamise teekonna. Kui lõikude arv on suurem kui 1, loob funktsioon kataloogi individuaalsete lõigufailide salvestamiseks. Seejärel iteriseerib ta läbi iga lõigu, konstrueerib Edge TTS käsu kasutades `calculate_rate_string()` funktsiooni, hääle nime ja lõigu teksti ning käivitab käsu kasutades `os.system()` funktsiooni. Kui käsu täitmine on edukas, lisab ta genereeritud audiofaili teekonna loendisse. Pärast kõigi lõikude töötlemist ühendab ta individuaalsed audiofailid funktsiooni `merge_audio_files()` abil ja salvestab liidetud audio määratud `save_path` teekonnale. Kui lõik on ainult üks, genereerib ta otse Edge TTS käsu ja salvestab audio `save_path`-i. Lõpuks tagastab see genereeritud audiofaili salvestusteekonna.

6. `random_audio_name_generate()`: See funktsioon genereerib juhusliku audiofaili nime kasutades [`uuid`] moodulit. See genereerib juhusliku UUID-i, teisendab selle stringiks, võtab esimesed 8 märki, lisab ".mp3" laienduse ja tagastab juhusliku audiofaili nime.

7. `talk(input_text)`: See funktsioon on peamine sisenemispunkt TTS-operatsiooni teostamiseks. See võtab parameetrina sisendi teksti. Esiteks kontrollib ta sisendi teksti pikkust, et määrata, kas tegemist on pika lausungiga (rohkem või võrdne 600 märgiga). Sõltuvalt pikkusest ja `translate_text_flag` muutuja väärtusest määrab ta keele ja genereerib tekstilõikude loendi kasutades `make_chunks()` funktsiooni. Seejärel genereerib ta audiofaili salvestusteekonna kasutades `random_audio_name_generate()` funktsiooni. Lõpuks kutsub ta välja `edge_free_tts()` funktsiooni TTS-operatsiooni teostamiseks ja tagastab genereeritud audiofaili salvestusteekonna.

Kokkuvõttes töötavad need funktsioonid koos, et jagada sisendi tekst lõikudeks, genereerida audiofaili nimi, teostada TTS-operatsioon Edge TTS teenuse abil ja ühendada individuaalsed audiofailid üheks lõplikuks audiofailiks.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Kahe funktsiooni implementatsioon: convert_to_text ja run_text_prompt, samuti kahe klassi deklaratsioon: str ja Audio.

The convert_to_text function takes an audio_path as input and transcribes the audio to text using a model called whisper_model. The function first checks if the gpu flag is set to True. If it is, the whisper_model is used with certain parameters such as word_timestamps=True, fp16=True, language='English', and task='translate'. If the gpu flag is False, the whisper_model is used with fp16=False. The resulting transcription is then saved to a file named 'scan.txt' and returned as the text.

The run_text_prompt function takes a message and a chat_history as input. It uses the phi_demo function to generate a response from a chatbot based on the input message. The generated response is then passed to the talk function, which converts the response into an audio file and returns the file path. The Audio class is used to display and play the audio file. The audio is displayed using the display function from the IPython.display module, and the Audio object is created with the autoplay=True parameter, so the audio starts playing automatically. The chat_history is updated with the input message and the generated response, and an empty string and the updated chat_history are returned.

The str class is a built-in class in Python that represents a sequence of characters. It provides various methods for manipulating and working with strings, such as capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, and more. These methods allow you to perform operations like searching, replacing, formatting, and manipulating strings.

The Audio class is a custom class that represents an audio object. It is used to create an audio player in the Jupyter Notebook environment. The class accepts various parameters such as data, filename, url, embed, rate, autoplay, and normalize. The data parameter can be a numpy array, a list of samples, a string representing a filename or URL, or raw PCM data. The filename parameter is used to specify a local file to load the audio data from, and the url parameter is used to specify a URL to download the audio data from. The embed parameter determines whether the audio data should be embedded using a data URI or referenced from the original source. The rate parameter specifies the sampling rate of the audio data. The autoplay parameter determines whether the audio should start playing automatically. The normalize parameter specifies whether the audio data should be normalized (rescaled) to the maximum possible range. The Audio class also provides methods like reload to reload the audio data from file or URL, and attributes like src_attr, autoplay_attr, and element_id_attr to retrieve the corresponding attributes for the audio element in HTML.

Overall, these functions and classes are used to transcribe audio to text, generate audio responses from a chatbot, and display and play audio in the Jupyter Notebook environment.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
Vastutusest loobumine:
See dokument on tõlgitud tehisintellekti tõlke­teenuse Co-op Translator (https://github.com/Azure/co-op-translator) abil. Kuigi me püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument selle algkeeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul on soovitatav kasutada professionaalset inimtõlget. Me ei vastuta võimalike arusaamatuste ega valesti tõlgendamise eest, mis sellest tõlkest võivad tuleneda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
